<a href="https://colab.research.google.com/github/mlsborges/imersao_ia_alura_google/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto: Perguntando sobre estatistica !!  



In [139]:
# Importando as bibliotecas e aplicando a API
import PyPDF2
import pandas as pd
import numpy as np
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
GOOGLE_API_KEY= api_key
genai.configure(api_key=GOOGLE_API_KEY)

## Extraindo o conteudo da consulta e colocando em um dataframe.<P>
Livro disponível disponível na internet no site: https://www.ufsm.br/app/uploads/sites/413/2018/11/04_estatistica.pdf


In [97]:
def extrair_paragrafos(pdf_file_path):
  paragrafos = []
  with open(pdf_file_path, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    for page_num in range(len(pdf_reader.pages)):
      page = pdf_reader.pages[page_num]
      page_text = page.extract_text()

      linhas = page_text.splitlines()

      paragrafo_atual = ""
      for linha in linhas:
        linha = linha.strip()
        if linha:
          paragrafo_atual += linha + " "
        else:
          if paragrafo_atual:
            paragrafos.append(paragrafo_atual.strip())
            paragrafo_atual = ""

      if paragrafo_atual:
        paragrafos.append(paragrafo_atual.strip())

  return paragrafos


In [98]:
pdf_file_path = '/content/Estatistica_PauloRobertodaCosta.pdf'
paragrafos = extrair_paragrafos(pdf_file_path)

In [99]:
df = pd.DataFrame(paragrafos, columns=['Paragrafos'])
df.head(10)

,Paragrafos
0,Estatística Paulo Roberto da Costa 2011Santa M...
1,"C837e Costa, Paulo Roberto da Estatística ..."
2,e-Tec Brasil 3 3Apresentação e-Tec Brasil Prez...
3,e-Tec Brasil 5Indicação de ícones Os ícones sã...
4,Tecnologia da Informática e-Tec Brasil 6
5,e-Tec Brasil 7Sumário Palavra do professor-aut...
6,e-Tec Brasil5.4 Distribuição de frequência 4...
7,e-Tec Brasil 9Palavra do professor-autor “Não ...
8,e-Tec Brasil 11Apresentação da disciplina Inve...
9,"posteriormente, se possa extrair o máximo de i..."


## Criando os embeddings para o conteúdo da consulta

In [100]:
model = "models/embedding-001"

In [110]:
def embed_fn(Paragrafos):
  return genai.embed_content(model=model,
                                 content=Paragrafos,)["embedding"]

In [113]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Paragrafos"]), axis=1)
df.head()

,Paragrafos,Embeddings
0,Estatística Paulo Roberto da Costa 2011Santa M...,"[0.03621656, -0.061976112, -0.060311902, 0.039..."
1,"C837e Costa, Paulo Roberto da Estatística ...","[0.016915306, -0.08009929, -0.019492123, -0.00..."
2,e-Tec Brasil 3 3Apresentação e-Tec Brasil Prez...,"[0.05264369, -0.014378905, -0.004100534, 0.031..."
3,e-Tec Brasil 5Indicação de ícones Os ícones sã...,"[0.031534314, -0.026692225, 0.0046140957, 0.02..."
4,Tecnologia da Informática e-Tec Brasil 6,"[0.027483173, 0.0051744343, -0.012294184, 0.01..."


# Função para realizar a consulta

In [117]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Paragrafos"]

# Estatistica formal

In [140]:
consulta = "Fale sobre medidas estatisticas"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

As medidas de tendência central mais utilizadas são: média aritmética, moda e mediana. Outras promédias menos usadas são as médias: geomé-trica, harmônica, quadrática, cúbica e biquadrática.saiba mais sobre estatística em: http://www.intervox.nce.ufrj.br/~diniz/d/direito/ou-estatistica.doc e-Tec Brasil Aula 7 - Medidas descritivas 67


# Estatistica descontraída

In [141]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config={"temperature": 1,  "candidate_count": 1})
response = model_2.generate_content(prompt)
print(response.text)

E aí, galera da estatística!

Temos uns bagulhos aqui chamados "medidas de tendência central". São eles que dão uma ideia geral dos seus dados.

Os mais famosos são:

* **Média aritmética:** A média que a gente usa no dia a dia. Soma todos os números e divide pela quantidade.
* **Moda:** O número que aparece mais vezes.
* **Mediana:** O número do meio, quando você coloca os dados em ordem crescente.

Tem umas outras médias menos usadas, mas não vamos entrar nessa fria agora, já deu pra sacar né?
